In [9]:
import tweepy
import requests
from PIL import Image
import io
import torch
from torchvision.transforms import ToTensor
from ipynb.fs.full.cnn import FashionCNN  # Import the CNN model

Epoch 1/10, Loss: 0.5189
Epoch 2/10, Loss: 0.3401
Epoch 3/10, Loss: 0.2939
Epoch 4/10, Loss: 0.2633
Epoch 5/10, Loss: 0.2381
Epoch 6/10, Loss: 0.2155
Epoch 7/10, Loss: 0.1981
Epoch 8/10, Loss: 0.1803
Epoch 9/10, Loss: 0.1639
Epoch 10/10, Loss: 0.1490


In [12]:
# https://developer.twitter.com/en/portal/projects-and-apps
# Authenticate and set up access to the Twitter API
consumer_key = "x"
consumer_secret = "x"
access_token = "x-x"
access_token_secret = "x"

In [14]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [13]:
# Load the saved CNN model
model = FashionCNN()
model.load_state_dict(torch.load("fashion_cnn.pth"))
model.eval()

FashionCNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=1600, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [15]:
# WILL NEED TOKENS AND PROPER AUTHORIZATION FOR THIS TO WORK

# Twitter @s to search from
accounts = ["Zara", "sheinofficial", "hm"]
hashtags = []

# Retrieve and classify images from Twitter
for account in accounts:
    tweets = api.user_timeline(screen_name=account, count=10, include_rts=False, tweet_mode="extended")
    for tweet in tweets:
        if 'media' in tweet.entities:
            for media in tweet.entities['media']:
                if media['type'] == 'photo':
                    image_url = media['media_url']
                    response = requests.get(image_url)
                    image = Image.open(io.BytesIO(response.content)).convert("RGB")
                    image_tensor = ToTensor()(image).unsqueeze(0)

                    # Classify the clothing type using the CNN model
                    output = model(image_tensor)
                    _, predicted = torch.max(output.data, 1)
                    predicted_class = predicted.item()

                    print(f"Tweet ID: {tweet.id}, Account: {account}, Predicted Class: {predicted_class}")

Unauthorized: 401 Unauthorized
89 - Invalid or expired token.